In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Function to load images from a directory
def load_images_from_folder(folder, label):
    if not os.path.exists(folder):
        print(f"Directory {folder} does not exist.")
        return [], []

    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (64, 64))  # Resize image to 64x64
            images.append(img.flatten())    # Flatten the image to 1D array
            labels.append(label)
        else:
            print(f"Could not read image {img_path}")
    return images, labels

# Step 2: Load and Preprocess the Dataset
# Define the paths to the training and testing folders
train_normal_path = '/content/drive/MyDrive/project/train/N'
train_infarction_path = '/content/drive/MyDrive/project/train/M'
test_normal_path = '/content/drive/MyDrive/project/test/N/N'
test_infarction_path = '/content/drive/MyDrive/project/test/M/M'

# Load training data
train_normal_images, train_normal_labels = load_images_from_folder(train_normal_path, 0)  # 0 for normal
train_infarction_images, train_infarction_labels = load_images_from_folder(train_infarction_path, 1)  # 1 for myocardial infarction

# Load testing data
test_normal_images, test_normal_labels = load_images_from_folder(test_normal_path, 0)
test_infarction_images, test_infarction_labels = load_images_from_folder(test_infarction_path, 1)

# Combine the training data
X_train = np.array(train_normal_images + train_infarction_images)
y_train = np.array(train_normal_labels + train_infarction_labels)

# Combine the testing data
X_test = np.array(test_normal_images + test_infarction_images)
y_test = np.array(test_normal_labels + test_infarction_labels)

# Check shapes and contents of X_train, y_train, X_test, y_test
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# Step 3: Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Step 4: Evaluate the Model
# Evaluate on training data
y_train_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("\nTraining Accuracy:", train_accuracy)
print("Training Confusion Matrix:\n", confusion_matrix(y_train, y_train_pred))
print("Training Classification Report:\n", classification_report(y_train, y_train_pred))

# Evaluate on testing data
if len(X_test) > 0:
    y_test_pred = clf.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print("\nTesting Accuracy:", test_accuracy)
    print("Testing Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
    print("Testing Classification Report:\n", classification_report(y_test, y_test_pred))
else:
    print("No testing data loaded. Check your directory paths and ensure they contain images.")

# Step 5: Classify New Data (Optional)
def classify_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img = cv2.resize(img, (64, 64))  # Resize image to 64x64
        img = img.flatten().reshape(1, -1)  # Flatten and reshape for prediction
        prediction = clf.predict(img)
        return "Normal" if prediction == 0 else "Myocardial Infarction"
    else:
        return "Image not found"

# Example usage for real-time image path prediction
def predict_realtime_ecg(image_path):
    result = classify_image(image_path)
    return result

# Example usage
realtime_image_path = '/content/drive/MyDrive/ecg.jpeg'
# Replace with your real-time image path
prediction_result = predict_realtime_ecg(realtime_image_path)
print(f'\nThe given real-time ECG image is classified as: {prediction_result}')


Mounted at /content/drive
X_train shape: (5820, 4096), y_train shape: (5820,)
X_test shape: (2914, 4096), y_test shape: (2914,)

Training Accuracy: 1.0
Training Confusion Matrix:
 [[3248    0]
 [   0 2572]]
Training Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3248
           1       1.00      1.00      1.00      2572

    accuracy                           1.00      5820
   macro avg       1.00      1.00      1.00      5820
weighted avg       1.00      1.00      1.00      5820


Testing Accuracy: 1.0
Testing Confusion Matrix:
 [[ 813    0]
 [   0 2101]]
Testing Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       813
           1       1.00      1.00      1.00      2101

    accuracy                           1.00      2914
   macro avg       1.00      1.00      1.00      2914
weighted avg       1.00      1.00      1.00      291

In [ ]:
from google.colab import drive
drive.mount('/content/drive')